In [12]:
# run this same experiment again but instead of 2,0,0 -> 0,0,2 have 2,0,0 -> 0,1,1 this would be cool 
# declaring all dependancies    
import qutip as qt
import numpy as np
from quantum_logical.pulsesim import QuantumSystem, Pulse
from quantum_logical.pulsesim.mode import QubitMode, SNAILMode, CavityMode
from quantum_logical.pulsesim.build_hamiltonian import Build_hamiltonian
import matplotlib.pyplot as plt
from qutip import Options
from tqdm import tqdm_notebook as tqdm
from progressbar import progressbar
from multiprocessing.pool import ThreadPool, Pool
import threading

opts = Options(nsteps=1e6, atol=1e-8, rtol=1e-6)
p_bar = qt.ui.TextProgressBar()  # None

c:\Users\girgi\Desktop\Github\quantum_logical\venv\lib\site-packages\qutip\solver\options.py:16: FutureWarning: Dedicated options class are no longer needed, options should be passed as dict to solvers.
  warnings.warn(


In [13]:
# adding all of the necessary elements to the full creation of the system 
qubit_dim = 2

# frequencies 
w1_un = 4
w2_un = 6
w3_un = 4.6666666666666666
ws_un = 5.6666666666666666

# coupling strength coefficient 
g1s = 2 * np.pi * 0.05467
g2s = 2 * np.pi * 0.0435
g3s = 2 * np.pi * 0.04875

# create array of the coefficients should be useful 
gs = [g1s, g2s, g3s]

# unknown value as of now 
g3 = .1

In [14]:
# creating the qubits and the hilbert space
qubit1 = QubitMode(
    mode_type="Qubit", name="q1", dim=qubit_dim, freq=w1_un, alpha=-0.161, T1=1e2, T2=5e1
)
qubit2 = QubitMode(
    mode_type="Qubit", name="q2", dim=qubit_dim, freq=w2_un, alpha=-0.1275, T1=1e2, T2=5e1
)
qubit3 = QubitMode(
    mode_type="Qubit", name="q3", dim=qubit_dim, freq=w3_un, alpha=-0.160, T1=1e2, T2=5e1
)
qubits = [qubit1, qubit2, qubit3]
snail = SNAILMode(mode_type="Snail", name="s", freq=ws_un, g3=0.3, dim=10, T1=1e3, T2=5e2)
_couplings = {
    frozenset([qubit1, snail]): 2 * np.pi * 0.05467,
    frozenset([qubit2, snail]): 2 * np.pi * 0.0435,
    frozenset([qubit3, snail]): 2 * np.pi * 0.04875
}

qs = QuantumSystem(qubits + [snail], couplings=_couplings)

system = []
system.extend(qubits)
system.extend([snail])

In [15]:
# list of qubit coupling strengths that will be given
g13 = g23 = g12 = .01
gqs = [[qubit1.freq,g12,g13],[g12,qubit2.freq,g23],[g13,g23,qubit3.freq]]

# creating the list of alphas that will be given to us 
a1 = a2 = a3 = .1
alphas = [a1, a2, a3]

In [16]:
# building the hammiltonian up to the pump term 
# creating the linear parts of the hamiltonian  
# the qubit and snail separate terms
H_no_interaction = [(i.freq / (2 * np.pi)) * (qs.modes_a_dag[i] * qs.modes_a[i]) for i in system]

# creating the snail mixing with the qubits terms
H_snail_qubits = [gs[qubits.index(i)] * (qs.modes_field[snail] * qs.modes_field[i]) for i in qubits]

# adding the coupling terms between the qubits 
H_qubit_qubit = []

for i in range(len(qubits)):
    for j in range(len(qubits)):
        if j > i:
            H_qubit_qubit.append(gqs[i][j] * (qs.modes_field[qubits[i]] * qs.modes_field[qubits[j]]))

# creating the nonlinear parts of the hamiltonian 
# adding the snail third order term first 
H_snail_third = [g3 * ((qs.modes_field[snail]) ** 3)]

# add the qubit non-linearity 
H_qubit_fourth = [alphas[qubits.index(i)] * (qs.modes_field[i] ** 4) for i in qubits]

In [17]:
H_total_no_time = []
# adding the linear parts into the hamiltonian 
H_total_no_time.extend(H_no_interaction)
H_total_no_time.extend(H_snail_qubits)
H_total_no_time.extend(H_qubit_qubit)
# adding the non-linear parts 
H_total_no_time.extend(H_snail_third)
H_total_no_time.extend(H_qubit_fourth)

In [18]:
H_no_time = sum(H_total_no_time)
freq_list  = [qubit1.freq / (2 * np.pi), qubit2.freq / (2 * np.pi), qubit3.freq / (2 * np.pi), snail.freq / (2 * np.pi)] 

In [19]:
# Hamiltonian construction
H_no_interaction1 = [freq_list[system.index(i)] * (qs.modes_a_dag[i] * qs.modes_a[i]) for i in system]
H_qubit_snail1 = [gs[qubits.index(i)] * (qs.modes_field[snail] * qs.modes_field[i]) for i in qubits]
H_qubit_qubit1 = [gqs[i][j] * (qs.modes_field[system[i]] * qs.modes_field[system[j]]) if j < i else 0 for i in range(3) for j in range(3)]
# H_qubit_qubit = []

# for i in range(len(qubits)):
#     for j in range(len(qubits)):
#         if j > i:
#             H_qubit_qubit.append(gqq[i][j] * (qs.modes_field[qubits[i]] * qs.modes_field[qubits[j]]))

H_snail_third1 = [g3 * (qs.modes_field[snail] ** 3)]
H_qubit_fourth1 = [(alphas[qubits.index(i)]) * (qs.modes_field[i] ** 4) for i in qubits]

H_total_no_time1 = []
H_total_no_time1.extend(H_no_interaction1)
H_total_no_time1.extend(H_qubit_snail1)
H_total_no_time1.extend(H_qubit_qubit1)
H_total_no_time1.extend(H_qubit_fourth1)
H_total_no_time1.extend(H_snail_third1)

H_no_time1= sum(H_total_no_time1)

In [20]:
H_no_time == H_no_time1

True